In [ ]:
# init the downloading path
from pathlib import Path

save_dir = Path.home() / "module_results" / "se.plan"
save_dir.mkdir(exist_ok=True, parents=True)

save_dir.is_dir()

In [ ]:
# get the country list
import pandas as pd
from component import parameter as pm

lmic_list = pd.read_csv(pm.country_list)

lmic_list.head()

In [ ]:
# get the gadm country list
from sepal_ui import aoi
from sepal_ui import sepalwidgets as sw
import numpy as np
from tqdm import tqdm

aoi_model = aoi.AoiModel(alert=sw.Alert(), gee=False)

country_list = pd.read_csv(aoi_model.FILE[0])  # GADM based


def append_gdf(admin=None):
    """append the gdf of the differnet LMIC areas if possible"""

    # get all the sub administrative areas
    df = country_list[country_list.GID_0 == admin]  # only the featured country
    df = df.drop_duplicates(subset="GID_1")  # remove all GID_2

    # get all the admin numbers
    gdf = None
    for i, row in df.iterrows():

        admin = row.GID_1 if row.GID_1 and row.GID_1 == row.GID_1 else row.GID_0

        aoi_model.admin = admin
        aoi_model.set_object(method="ADMIN0")

        tmp_gdf = aoi_model.gdf
        gdf = tmp_gdf if type(gdf) == type(None) else gdf.append(tmp_gdf)

    return gdf

In [ ]:
import geopandas as gpd

gdf_adm1_path = save_dir / "lmic_leve1.shp"

if gdf_adm1_path.is_file():

    # read the file
    gdf = gpd.read_file(gdf_adm1_path)

else:
    gdf = None
    with tqdm(total=len(lmic_list)) as pbar:
        for i, row in lmic_list.iterrows():

            tmp_gdf = append_gdf(row.ISO3)
            gdf = tmp_gdf if type(gdf) == type(None) else gdf.append(tmp_gdf)

            pbar.update()

    gdf.to_file(gdf_adm1_path)

len(gdf)

In [ ]:
len(gdf.GID_0.unique()) == 139  ## check that all the countries are included

In [ ]:
# get all the layers
import ee
import pandas as pd

ee.Initialize()

from component import parameter as pm

layer_list = (
    pd.read_csv(pm.layer_list)
    .filter(items=["layer_id", "theme", "gee_asset"])
    .sort_values(by=["theme"])
)
layer_list.head()

In [ ]:
# create the reference parameters
name = "treecover_with_potential"
layer = layer_list[layer_list.layer_id == name]

ee_ref = ee.Image(layer.iloc[0].gee_asset)
ee_ref_crs = ee_ref.projection()
ee_ref_geom = ee_ref.geometry()

In [ ]:
# export all the layers witht the adapted reducer
from component.scripts import gdrive

drive_handler = gdrive()
with tqdm(total=len(layer_list)) as pbar:
    for i, row in layer_list.iterrows():

        ee_image = None
        ee_reducer = None
        if row.layer_id in ["protected_areas"]:  # rasterize before export

            ee_reducer = ee.Reducer.allNonZero()
            ee_image = (
                ee.FeatureCollection(row.gee_asset.strip())
                .filter(ee.Filter.neq("WDPAID", {}))
                .reduceToImage(properties=["WDPAID"], reducer=ee.Reducer.allNonZero())
                .gt(0)
                .unmask(0)
                .rename("wdpa")
                .select("wdpa")
                .reproject(ee_ref_crs)
            )

        elif row.layer_id in [
            "treecover_with_potential",
            "declining_population",
        ]:  # binaries inclusive

            ee_reducer = ee.Reducer.anyNonZero()

        elif row.layer_id in ["ecozones", "land_cover"]:  # most frequent value
            ee_reducer = "mode"

        ee_image = ee_image if ee_image else ee.Image(row.gee_asset.strip()).select(0)
        ee_reducer = ee_reducer if ee_reducer else ee.Reducer.mean()

        # export
        if (
            not len(drive_handler.get_files(row.layer_id))
            and not (save_dir / f"{row.layer_id}.vrt").is_file()
        ):
            task_config = {
                "folder": "se.plan",
                "image": ee_image.reduceResolution(reducer=ee_reducer, maxPixels=2048),
                "description": row.layer_id,
                "region": ee_ref_geom,
                "scale": 1000,
                "crs": ee_ref_crs,
                "maxPixels": 10e12,
            }

            task = ee.batch.Export.image.toDrive(**task_config)
            task.start()

        pbar.update()

print(
    "You can now monitor your exporting steps from earthegine: https://code.earthengine.google.com"
)

In [ ]:
# download treecover files
# add them in the same directory
# add them to the layer list

ee_treecover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select(
    "tree-coverfraction"
)

treecover = "curent_treecover"
# export
if (
    not len(drive_handler.get_files(treecover))
    and not (save_dir / f"{treecover}.vrt").is_file()
):
    task_config = {
        "folder": "se.plan",
        "image": ee_image,
        "description": treecover,
        "region": ee_ref_geom,
        "scale": 1000,
        "crs": ee_ref_crs,
        "maxPixels": 10e12,
    }

    task = ee.batch.Export.image.toDrive(**task_config)
    task.start()

# add to the layer_list
if not treecover in layer_list.layer_id:
    layer_list.loc[len(layer_list)] = [treecover, "treecover", ""]

In [ ]:
# once the image are available on my drive
# download theme as files
from pathlib import Path
from osgeo import gdal

raw_dir = save_dir / "raw_data"
raw_dir.mkdir(exist_ok=True)

with tqdm(total=len(layer_list)) as pbar:
    for i, row in layer_list.iterrows():

        vrt_path = save_dir / f"{row.layer_id}.vrt"

        if vrt_path.is_file():
            pbar.update()
            continue

        files = drive_handler.get_files(row.layer_id)
        if len(files):
            loc_files = drive_handler.download_files(files, raw_dir)
            drive_handler.delete_files(files)

            # create a vrt to manipulate everything
            ds = gdal.BuildVRT(str(vrt_path), [str(f) for f in loc_files])
            ds.FlushCache()

        pbar.update()

In [ ]:
# add the bastin dataset
# add the dataset to the raw folder under total_potential.tif file
import rasterio as rio
from rasterio import warp

# align the dataset on the others
def align(src_rst, template_rst, out_rst, verbose=False):
    """Align a source raster on a template

    Args :
        src_rst (str) : path to the source raster
        template_rst (str) : path to the template raster
        out_rst (str) : path to the output raster

    Return :
        out_rst

    """

    # get template crs and transform
    with rio.open(template_rst) as tplt, rio.open(src_rst) as src:

        transform, width, height = warp.calculate_default_transform(
            src.crs, tplt.crs, tplt.width, tplt.height, *tplt.bounds
        )

        kwargs = src.meta.copy()

        kwargs.update(
            driver="GTiff",
            height=height,
            width=width,
            transform=transform,
            compress="lzw",
        )

        # destination
        with rio.open(out_rst, "w", **kwargs) as dst:
            for i in range(1, dst.count + 1):
                warp.reproject(
                    source=rio.band(src, i),
                    destination=rio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=tplt.crs,
                    resampling=warp.Resampling.bilinear,
                )

    print(f"The raster {src_rst} has been align on {template_rst} in {out_rst}")

    return


name = "potential_treecover"
input_tif = raw_dir / "Total_potential.tif"
output_tif = raw_dir / f"{name}.tif"
template_vrt = save_dir / "treecover_with_potential.vrt"

if not output_tif.is_file():
    align(input_tif, template_vrt, output_tif)

    # create a binding vrt
    vrt_path = save_dir / f"{name}.vrt"
    ds = gdal.BuildVRT(str(vrt_path), [str(output_tif)])
    ds.FlushCache()

if not name in layer_list.layer_id:
    layer_list.loc[len(layer_list)] = [name, "treecover", ""]

In [ ]:
len(layer_list) == 25  # initial number + 2

In [ ]:
# gather all the layers into 1 single vrt
global_vrt_path = save_dir / "layers.vrt"

layer_files = [
    str(save_dir / f"{row.layer_id}.vrt") for _, row in layer_list.iterrows()
]
ds = gdal.BuildVRT(str(global_vrt_path), layer_files, separate=True)
ds.FlushCache()

In [ ]:
import rasterio as rio

# read the mask layer
with rio.open(save_dir / "treecover_with_potential.vrt") as mask_f:
    mask = mask_f.read(1)

    # count the number of unmask values
    nb_unmasked = np.sum(mask)

nb_unmasked

In [ ]:
import rasterio as rio
from rasterio.windows import Window
from itertools import product
from shapely import geometry as sg
import time

import warnings

warnings.filterwarnings("ignore")

# create the csv
csv_f = save_dir / "world_Data_restoration_1km.csv"

if not csv_f.is_file():

    # read the mask layer
    with rio.open(save_dir / "treecover_with_potential.vrt") as mask_f:
        mask = mask_f.read(1)

        # count the number of unmask values
        nb_unmasked = np.sum(mask)

        # init the columns
        # allocate the array for efficiency
        columns = layer_list.layer_id.to_list()
        data = {
            "long": np.empty(nb_unmasked),
            "lat": np.empty(nb_unmasked),
            "ADM_0": np.empty(nb_unmasked, dtype=str),
            "ADM_1": np.empty(nb_unmasked, dtype=str),
        }

        with tqdm(total=mask_f.height * mask_f.width) as pbar:
            for i, xy in enumerate(product(range(mask_f.height), range(mask_f.width))):

                # expand
                x, y = xy

                # don't do anything if it's masked
                if mask[x][y] == 0:

                    data["long"][i] = np.nan
                    data["lat"][i] = np.nan

                    data["ADM_0"][i] = ""
                    data["ADM_1"][i] = ""

                else:

                    # get the coordinates
                    coords = rio.transform.xy(
                        mask_f.profile["transform"], x, y, offset="center"
                    )  # x is lat
                    data["long"][i] = coords[1]
                    data["lat"][i] = coords[0]

                    # get the administrative values
                    point = sg.Point(coords[::-1])
                    min_ = gdf.distance(point).min()
                    row = gdf[gdf.distance(point) == min_].iloc[0]
                    data["ADM_0"][i] = row.NAME_0
                    data["ADM_1"][i] = row.NAME_1

                pbar.update()

In [ ]:
import rasterio as rio
import rioxarray
import pandas
import numpy as np

# if not csv_f.is_fiel():

# add data to the data dict
mask = rioxarray.open_rasterio(
    save_dir / "treecover_with_potential.vrt", chunks=5
).astype(int)
print(mask.shape)

for _, row in layer_list.iterrows():
    rds = rioxarray.open_rasterio(save_dir / f"{row.layer_id}.vrt", chunks=5)
    rds.drop("spatial_ref").drop("band")
    rds = rds.where(mask == 1)
    rds.name = row.layer_id
    df = rds.to_dataframe().reset_index()

    break

df.head()

In [ ]:
import rasterio as rio
from rasterio.windows import Window
from itertools import product
from shapely import geometry as sg
import time

import warnings

warnings.filterwarnings("ignore")

# create the csv
csv_f = save_dir / "world_Data_restoration_1km.csv"

if not csv_f.is_file():

    # read the mask layer
    with rio.open(save_dir / "treecover_with_potential.vrt") as mask_f:
        mask = mask_f.read(1)

        # count the number of unmask values
        nb_unmasked = np.sum(mask)

        # init the columns
        # allocate the array for efficiency
        columns = layer_list.layer_id.to_list()
        data = {
            "long": np.empty(nb_unmasked),
            "lat": np.empty(nb_unmasked),
            "ADM_0": np.empty(nb_unmasked, dtype=str),
            "ADM_1": np.empty(nb_unmasked, dtype=str),
        }
        data = {**data, **{k: np.empty(nb_unmasked) for k in columns}}

        with tqdm(total=mask_f.height * mask_f.width) as pbar:
            count = 0
            for x, y in product(range(mask_f.height), range(mask_f.width)):

                # don't do anything if it's masked
                if mask[x][y] != 0:

                    start = time.time()
                    # get the coordinates
                    coords = rio.transform.xy(
                        mask_f.profile["transform"], x, y, offset="center"
                    )  # x is lat
                    data["long"][count] = coords[1]
                    data["lat"][count] = coords[0]
                    print(f"get coords: {time.time()-start}s")

                    start = time.time()
                    # get the administrative values
                    point = sg.Point(coords[::-1])
                    min_ = gdf.distance(point).min()
                    row = gdf[gdf.distance(point) == min_].iloc[0]
                    data["ADM_0"][count] = row.NAME_0
                    data["ADM_1"][count] = row.NAME_1
                    print(f"get admin: {time.time()-start}s")

                    start_layer = time.time()
                    # get the value of all the other layers
                    window = Window(y, x, 1, 1)
                    with rio.open(global_vrt_path) as f:

                        for i, column in enumerate(columns):
                            data[column][count] = f.read(i + 1, window=window)[0][0]

                    print(f"get all layers: {time.time()-start_layer}s")

                    # update the count value
                    count += 1

                    break

                pbar.update()

data

In [ ]:
# create the df from it
if not csv_f.is_file():
    df = pd.DataFrame(data)
else:
    df = pd.read_csv(csv_f)

df.head()

In [ ]:
df.to_csv(save_dir / "world_Data_restoration_1km.csv")
# df.to_csv(csv_f)